<a href="https://colab.research.google.com/github/elio-li/CSCI-4964/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

## Part a) Build a bipartite network

To construct a adjacency matrix, we use the following strategy:

```
      1428         456
|--------------|---------|
|              |         |
|              |         |
|      0       |    M    |  1428
|              |         |
|              |         |
|--------------|---------|
|              |         |
|     M^T      |    0    |  456
|              |         |
|--------------|---------|
```

where `M` is a 1428 * 456 Matrix(which is the dataset given)

In [ ]:
import numpy as np

file = open("networkdata1.csv") # changed the file to csv
upper_right = np.loadtxt(file, delimiter=",")
lower_left = upper_right.T

In [ ]:
upper_left = np.zeros((1428,1428))
lower_right = np.zeros((456,456))
left = np.concatenate((upper_left, lower_left), axis=0)
right = np.concatenate((upper_right, lower_right), axis=0)
matrix = np.concatenate((left, right), axis=1)

## Part b) Build the projection networks of plants and animals

Algorithm as follows(using animal projection network for example):

```
for each `x` in 1428 animals:
  for each `y` in 456 plants:
    if the interaction is 1:
      transpose matrix, rows will become plants'
      look for all the 1's from the corresponding plant array
        for each 1 is found, two animals connect to the same plant
set all the diagonal of the result to 0 at the end
```

In [ ]:
# data -> input matrix (1428 * 456 for this homework)
# size -> either m or n from m*n, this is to determine
#          finding which projection network
def find_projection(data, size):
  proj = np.zeros((size, size))
  for x in range(len(data)):
    for y in range(len(data[0])):
      if data[x,y] == 1:
        for z in range(len(data)):
          if data.T[y][z] == 1:
            proj[x][z] = 1
  for i in range(size):
    proj[i][i] = 0
  return proj

In [ ]:
animal_proj = find_projection(upper_right, 1428)
plants_proj = find_projection(upper_right.T, 456)

## Part c) Calculate the degree of each nodes in the projection networks

This part we can just sum up each rows of both projection matrices

In [ ]:
animal_degr = np.zeros(1428)
plants_degr = np.zeros(456)

for i in range(len(animal_proj)):
  animal_degr[i] = np.sum(animal_proj[i])
for i in range(len(plants_proj)):
  plants_degr[i] = np.sum(plants_proj[i])

animal_degr

## Part d) Plot the degree distributions of each projection network

In [ ]:
animal_degr_count = np.zeros(1428)
plants_degr_count = np.zeros(456)

for i in range(len(animal_degr)):
  animal_degr_count[int(animal_degr[i])] += 1
for i in range(len(plants_degr)):
  plants_degr_count[int(plants_degr[i])] += 1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(animal_degr_count/1428)

In [ ]:
plt.plot(plants_degr_count/456)

## Part e) Find three pair of nodes with distance 6 (shortest path).

In [ ]:
def BFS(matrix, root, iteration = -1):
  visited = [0 for i in range(len(matrix))]
  path = []
  queue = []
  clock = 0

  def find_neighbors(i):
    for j in range(len(matrix)):
      if matrix[i][j] == 1 and visited[j] == 0:
        return j
    return -1

  # start:
  visited[root] = 1
  queue.append(root)
  # bfs until queue is empty or 6 is reached
  while (iteration == -1 and len(queue) != 0) or (iteration != -1 and clock != iteration and len(queue) != 0):
    cur_vertex = queue.pop()
    path.append(cur_vertex)
    nei_vertex = find_neighbors(cur_vertex)
    while nei_vertex != -1:
      visited[nei_vertex] = 1
      queue.append(nei_vertex)
      nei_vertex = find_neighbors(cur_vertex)
    clock += 1
  return path

In [ ]:
path1 = BFS(plants_proj, 0, 6)
path2 = BFS(plants_proj, 50, 6)
path3 = BFS(plants_proj, 100, 6)

print(path1,path2,path3)

In [ ]:
path1 = BFS(animal_proj, 0, 6)
path2 = BFS(animal_proj, 300, 6)
path3 = BFS(animal_proj, 800, 6)

print(path1,path2,path3)

The above are 6 lists that contains 6 nodes each, the first node in each list is the starting point, the last node is the endpoint.

## Part f) Calculate the diameter of each projection network.

## Part g) Find the largest connected component of each projection network

In [ ]:
def connected_component(matrix):
  labels = [0 for i in range(len(matrix))]
  label = 1
  # find a node that haven't walked
  while 0 in labels:
    i = next((i for i, x in enumerate(labels) if x == 0), None)
    if i == None:
      break
    nodes = BFS(matrix, i)
    for j in nodes:
      labels[j] = label
    label += 1
  return labels

In [ ]:
def find_largest_connected(List):
  counter = 0
  num = List[0]
     
  for i in List:
    curr_frequency = List.count(i)
    if curr_frequency > counter:
      counter = curr_frequency
      num = i
 
  return num

In [ ]:
plants_connected_list = connected_component(plants_proj)

In [ ]:
plants_connected_max = find_largest_connected(plants_connected_list)
plants_max_indices = [i for i, x in enumerate(plants_connected_list) if x == plants_connected_max]

In [ ]:
animal_connected_list = connected_component(animal_proj)

In [ ]:
animal_connected_max = find_largest_connected(animal_connected_list)
animal_max_indices = [i for i, x in enumerate(animal_connected_list) if x == animal_connected_max]

## Part h) Remove nodes 1 to 300 from the the pollinator projection network, recalculate its largest connected component.

In [ ]:
new_grid_h = upper_right[300:]
animal_proj_h = find_projection(new_grid_h, 1128)
animal_connected_list_h = connected_component(animal_proj_h)
animal_connected_max_h = find_largest_connected(animal_connected_list_h)
animal_max_indices_h = [i for i, x in enumerate(animal_connected_list_h) if x == animal_connected_max_h]

## Part i) Remove nodes 1 to 100 from the the flower projection network, recalculate its largest connected component.

In [ ]:
new_grid_i = upper_right.T[100:]
plants_proj_i = find_projection(new_grid_i, 356)
plants_connected_list_i = connected_component(plants_proj_i)
plants_connected_max_i = find_largest_connected(plants_connected_list_i)
plants_max_indices_i = [i for i, x in enumerate(plants_connected_list_i) if x == plants_connected_max_i]

## MISC.

In [ ]:
a = np.array(animal_max_indices_h)
mat = np.matrix(a)
with open('i)plants_connected_component.txt','wb') as f:
    for line in mat:
        np.savetxt(f, line, fmt='%d')